In [1]:
def Validation(n_fold,X):
    list_train_fold = []
    list_val_fold = []
    list_train = []
    Number = X.shape[0]//n_fold
    for i in range(X.shape[0]):
        list_train.append(i)
    
    for i in range(n_fold)[::-1]:
        list_val = []
        if i==n_fold-1:
            for j in range(Number*i,X.shape[0]):
                list_val.append(j)
                
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
            
        
        if i != n_fold-1: 
            for j in range(Number*i,Number*(i+1)):
                list_val.append(j)
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
        
    return list_train_fold, list_val_fold

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')
from lib import models_update, graph, coarsening, utils
# modified
import tensorflow.compat.v1 as tf
import numpy as np
import time
import pandas as pd
%matplotlib inline
import random
import tensorflow as tf
tf.__version__


'1.15.0'

In [3]:
def data_simulation(exp, ppi,p):
    dataset = {}
    all_nodes = exp.shape[1]
    dataset['exp'] = []
    dataset['L'] = []
    dataset['perm'] = []
    Pathway_Name = list(p)
    
        
    for i in range(len(Pathway_Name)):
        
    
        Node_Number = []
        Sub_Pathway = p[Pathway_Name[i]]['gene'].values() #i번째 Pathway의 GeneExpression 저장
        Numbering = list(Sub_Pathway) # Gene 갯수 체크
        for a in range(len(Numbering)): 
            for b in range(exp.shape[1]): 
                if Numbering[a] == exp.columns[b]: # 해당하는 Gene이름의 index 번호 체크
                    Node_Number.append(b)

        sample_ppi = ppi.iloc[Node_Number, Node_Number] # ppi를 해당하는 Gene과 맞춤
        sample_ppi = np.array(sample_ppi) 
        print("PPI Shape :",len(Node_Number), sample_ppi.shape[0])
        A = coo_matrix(sample_ppi,dtype=np.float32) 
        print("Number of Edge :",A.nnz//2, "//",i)
        
        graphs, perm = coarsening.coarsen(A, levels=4, self_connections=False)
        print(type(A))
        L = [graph.laplacian(A, normalized=True) for A in graphs]
        dataset['exp'].append(exp.iloc[:, Node_Number].values) 
        dataset['L'].append(L)
        dataset['perm'].append(perm)
    return dataset

In [39]:
import pickle
from scipy.sparse import coo_matrix
ppi = pd.read_csv("Table/PPI.csv")
ppi.drop(['string'], axis='columns', inplace=True)
exp = pd.read_csv("Table/EXP.csv")
exp.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(exp.shape)
print(ppi.shape)
with open('Table/Pathway.pkl','rb')as f: #Pathway 받아오기
    p = pickle.load(f)



(734, 7545)
(7545, 7545)


In [ ]:
dataset = data_simulation(exp, ppi,p)

In [5]:
data_IC50 = pd.read_csv("Data/Table_S6_GDSC_Drug_response_IC50.csv")
data_IC50.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(data_IC50.shape)

(734, 201)


In [36]:
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
n_fold = 3
Y_train, Y_test = train_test_split(data_IC50,  test_size=0.25, shuffle=True, random_state=20)


for cv in range(1):
    Y_pred = np.zeros([Y_test.shape[0], Y_test.shape[1]])
    Y_test = np.zeros([Y_test.shape[0], Y_test.shape[1]])
#    print('n_fold:{}'.format(cv))
    j = 0
    for i in range(1):
        label = data_IC50.iloc[:,i]
        label = np.array(label)
        data_minmax = label[~np.isnan(label)]
        min = data_minmax.min()
        max = data_minmax.max()
        label = (label - min) / (max - min)
#         label_indexes = np.argwhere([~np.isnan(label)])[:, 1]
        
        print(label.shape[0])
        train_idx, test_idx = train_test_split(range(label.shape[0]), test_size=0.25, shuffle=True, random_state=20)
        train_idx = np.array(train_idx)
        print(train_idx.shape)
        train_idx = train_idx[~np.isnan(label[train_idx])]        
        list_train, list_val = Validation(n_fold,train_idx)

        val_idx = train_idx[list_val[cv]]
        print(train_idx.shape)
        train_idx = train_idx[list_train[cv]]
        print(train_idx.shape)
        test_idx = np.array(test_idx)
        
        train_labels = label[train_idx]
        val_labels = label[val_idx]
        test_labels = label[test_idx]
        
        
        common = {}
        common['num_epochs']     = 40
        common['batch_size']     = 4
        common['decay_steps']    = train_idx.shape[0] / common['batch_size']
        common['eval_frequency'] = 10 * common['num_epochs']
        common['brelu']          = 'b1relu'
        common['pool']           = 'mpool1'

        common['regularization'] = 0
        common['dropout']        = 1
        common['learning_rate']  = 0.02
        common['decay_rate']     = 0.95
        common['momentum']       = 0.9
        common['F']              = [1]
        common['K']              = [1]
        common['p']              = [2]
        common['M']              = [1]
        # concatnate fully connected NN
        common['M1']             = [1]

        train_dataset = []
        val_dataset = []
        test_dataset = []
        # 3개의 dataset을 각 pathway에 맞게 연결
        for kegg_id in range(len(dataset['L'])):
            exp = dataset['exp'][kegg_id]
            perm = dataset['perm'][kegg_id]
            train_data = exp[train_idx, :]
            val_data = exp[val_idx, :]
            test_data = exp[test_idx, :]
            train_data = coarsening.perm_data(train_data, perm)
            val_data = coarsening.perm_data(val_data, perm)
            test_data = coarsening.perm_data(test_data, perm)
            train_dataset.append(train_data)
            val_dataset.append(val_data)
            test_dataset.append(test_data)
        if True:
            name = 'cgconv_softmax'
            params = common.copy()
        
        
        model = models_update.cgcnn(dataset['L'], **params)
        loss, t_step = model.fit(train_dataset, train_labels, val_dataset, val_labels)


        Y_pred[:, j] = model.predict(test_dataset)
        Y_test[:, j] = test_labels        
        print(Y_pred[:, j])
        print(Y_test[:, j])
        j = j+1
#       np.savez(('parameter.cv_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

    np.savez(('parameter.cv_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

        


734
(550,)
(543,)
(362,)


In [47]:
from scipy import stats
RMSE = []
PCC = []
SCC = []

for i in range(4):
    y_test = Y_test[:, i]
    y_pred = Y_pred[:, i]
    y_pred = y_pred[~np.isnan(y_test)]
    y_test = y_test[~np.isnan(y_test)]

    RMSE.append(mean_squared_error(y_test,y_pred))
    PCC.append(stats.pearsonr(y_test,y_pred)[0])
    SCC.append(stats.spearmanr(y_test,y_pred)[0])
print("RMSE :",RMSE)
print("PCC :",PCC)
print("SCC :",SCC)


RMSE : [0.03449682940019994, 0.016506825518722328, 0.03304470752500458, 0.0359634220979898]
PCC : [0.3829299218333685, 0.41952147256776295, 0.23990148477324408, 0.27363753511941785]
SCC : [0.3694496743726658, 0.4427821974239871, 0.23844460592236225, 0.2894369167772668]
